## ETL Run Notebook  - Run scripts & Test w/ sql queries

### Run create_table.py to see if tables can be created property

- Make sure you open a RedShift cluster and make it publicly accessible

In [ ]:
try:
    %run create_tables.py 
except Exception as e:
    print(e)

In [ ]:
try:
    %run etl.py 
except Exception as e:
    print(e)

#### Test tables with some Queries